In [8]:
# first couple of cells are the same as previous notebooks
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
passwords = open('rockyou.txt', 'r',encoding='latin1').read().splitlines()

In [10]:
allowed = set("abcdefghijklmnopqrstuvwxyz0123456789") # for simplification
filtered_passwords = []
for p in passwords:
    if all(char in allowed for char in p):
        filtered_passwords.append(p)

In [11]:
chars =  ["."] + sorted(list(set(''.join(filtered_passwords))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
stoi

{'.': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36}

In [12]:
seq_length = 4  # context length: how many characters do we take to predict the next one?

In [13]:
# training data setup -  function generalization based on context length
def create_sequences(set_size,seq_length):
    inputs = []
    targets = []
    for password in filtered_passwords[:set_size]:
        password = '.' + password + '.'
        for i in range(len(password) - seq_length):
            input_seq = password[i:i + seq_length]
            target_char = password[i + seq_length]
            inputs.append([stoi[char] for char in input_seq])
            targets.append(stoi[target_char])
    return np.array(inputs), np.array(targets)


X_train, y_train = create_sequences(50000,seq_length)

In [14]:
# parameters
input_size = len(chars)  # Number of unique characters
hidden_size = 256
output_size = len(chars)  
num_epochs = 100
learning_rate = 0.001

In [16]:
X_train = torch.tensor(X_train,dtype=torch.long)
y_train = torch.tensor(y_train)
X_one_hot = F.one_hot(X_train, num_classes=input_size).float()


C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\3888055159.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train,dtype=torch.long)
C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\3888055159.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train)


In [17]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)  # out shape: (batch_size, seq_length, hidden_size)
        out = self.fc(out[:, -1, :])  # Get the output of the last time step
        return out

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [21]:
X_one_hot = X_one_hot.to(device)
y_train = y_train.to(device)

In [22]:
model = SimpleRNN(input_size, hidden_size, output_size)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_one_hot)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'

In [ ]:
def generate_password(model, start_sequence, max_length=20):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to track gradients
        current_sequence = start_sequence
        generated_password = start_sequence

        for _ in range(max_length):
            # Prepare input
            x = torch.tensor([[stoi[c] for c in current_sequence]], dtype=torch.long)
            x_one_hot = F.one_hot(x, num_classes=input_size).float()
            
            # Forward pass
            output = model(x_one_hot)
            
            # Get probabilities
            probs = F.softmax(output, dim=1)
            
            # Sample next character
            next_char_index = torch.multinomial(probs, 1).item()
            next_char = itos[next_char_index]
            
            # Stop if we generate the end token
            if next_char == '.':
                break
            
            # Add to generated password and update current sequence
            generated_password += next_char
            current_sequence = current_sequence[1:] + next_char

    return generated_password[1:]  # Remove the start token

# Example usage:
start_sequence = '.kyl'  # Start with a context of seq_length
for i in range(10):   
    generated_password = generate_password(model, start_sequence)
    print(generated_password)

